# Import Libraries

In [1]:
import pandas as pd 
import numpy as np 

# Preparing Data

Duplicate values in the Spotify dataset are chosen to be deleted as they are not very useful when duplicated.

In [2]:
df_spotify = pd.read_csv("../data/spotify_to_merge.csv")
df_spotify = df_spotify.drop_duplicates(subset='track_name', keep='first')
df_grammys = pd.read_csv("../data/grammys_to_merge.csv")
df_grammys.rename(columns={'artist': 'artists', 'nominee':'track_name'}, inplace=True)

# Merging Data


The columns 'artists' and 'track_name' are used as references when performing the merge. 

First, the names in the Grammy dataset are changed, and then the merge is carried out.

In [3]:
df_merged = pd.merge(df_grammys, df_spotify, on=["artists", "track_name"], how="outer")
df_merged

,track_name,artists,winner,awards_group,title_by_year,album_name,popularity,duration_ms,explicit,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,pagadoff,!nvite,NaN,NaN,NaN,pagadoff,5.0,135860.0,False,0.784,...,-7.591,0.0,0.3480,0.332,0.00362,0.1310,0.501,84.997,4.0,study
1,strolling,!nvite,NaN,NaN,NaN,strolling,41.0,138875.0,False,0.857,...,-12.755,1.0,0.1920,0.666,0.01910,0.1260,0.329,84.997,4.0,study
2,Going on a Mission,"""Puppy Dog Pals"" Cast",NaN,NaN,NaN,Puppy Dog Pals: Disney Junior Music,55.0,38144.0,False,0.629,...,-3.839,0.0,0.0470,0.021,0.00000,0.0930,0.957,93.937,3.0,children
3,Puppy Dog Pals Main Title Theme,"""Puppy Dog Pals"" Cast",NaN,NaN,NaN,Puppy Dog Pals: Disney Junior Music,60.0,57789.0,False,0.781,...,-4.709,1.0,0.2020,0.171,0.00141,0.2020,0.873,182.148,4.0,children
4,"Amish Paradise (Parody of ""Gangsta's Paradise""...","""Weird Al"" Yankovic",NaN,NaN,NaN,Bad Hair Day,58.0,202920.0,False,0.728,...,-10.540,1.0,0.1720,0.103,0.00000,0.2670,0.483,80.902,4.0,comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78322,NaN,NaN,True,Excellence Awards,(1971-1999) AGM,NaN,0.0,0.0,False,0.501,...,-9.460,0.0,0.0605,0.690,0.00396,0.0747,0.734,138.391,4.0,k-pop
78323,NaN,NaN,True,Excellence Awards,(1971-1999) AGM,NaN,0.0,0.0,False,0.501,...,-9.460,0.0,0.0605,0.690,0.00396,0.0747,0.734,138.391,4.0,k-pop
78324,NaN,NaN,True,Excellence Awards,(1971-1999) AGM,NaN,0.0,0.0,False,0.501,...,-9.460,0.0,0.0605,0.690,0.00396,0.0747,0.734,138.391,4.0,k-pop
78325,NaN,NaN,True,Excellence Awards,(1958-1970) AGM,NaN,0.0,0.0,False,0.501,...,-9.460,0.0,0.0605,0.690,0.00396,0.0747,0.734,138.391,4.0,k-pop


## Improving Merge Data

Now, looking at the dataframe after the merge, we see that there are many data points that are not useful, such as a large number of null values (which will be handled later), among other things.

By using dataframes, it is decided to start searching and selecting data that match in both datasets (Spotify and Grammys) in the columns of artists and track_name, which will be helpful when wanting to do a future analysis.

In [4]:
common_tracks = pd.merge(df_grammys[['track_name']], df_spotify[['track_name']], on='track_name', how='inner')
nominated_tracks = df_merged[df_merged['winner'] == True]
nominated_winner_tracks = nominated_tracks[['track_name']]
common_winner_tracks = pd.merge(common_tracks, nominated_winner_tracks, on='track_name', how='inner')

nominated_winner_tracks = nominated_tracks[['track_name', 'artists', 'awards_group']]
common_winner_tracks = pd.merge(common_tracks, nominated_winner_tracks, on='track_name', how='inner')


In [5]:
common_winner_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1357 entries, 0 to 1356
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   track_name    1321 non-null   object
 1   artists       946 non-null    object
 2   awards_group  1357 non-null   object
dtypes: object(3)
memory usage: 31.9+ KB


## Getting Data Ready. 

Once these similarities between the datasets are found, a final merge is decided, which will have all this data. 

In turn, due to the large number of nulls, it is decided to handle a maximum of 10,000 data with which a better analysis and management can be done. 

Finally, the most relevant columns are selected, leaving our final dataframe ready.

In [6]:
final_df = pd.merge(df_merged, common_winner_tracks, on=['track_name', 'artists', 'awards_group'], how='outer')
final_df = final_df.iloc[:10000]
final_df = final_df[['track_name', 'artists', 'winner', 'awards_group', 'title_by_year']]
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   track_name     10000 non-null  object
 1   artists        9541 non-null   object
 2   winner         1039 non-null   object
 3   awards_group   1039 non-null   object
 4   title_by_year  1039 non-null   object
dtypes: object(5)
memory usage: 390.8+ KB


In [7]:
final_df

,track_name,artists,winner,awards_group,title_by_year
0,!I'll Be Back!,Rilès,NaN,NaN,NaN
1,"""A"" You're Adorable",Brian Hyland,NaN,NaN,NaN
2,"""C"" IS FOR COOKIE",Little Apple Band,NaN,NaN,NaN
3,"""C"" is for Cookie",Little Apple Band,NaN,NaN,NaN
4,"""Christe, Redemptor omnium""",Traditional;Sistine Chapel Choir;Massimo Palom...,NaN,NaN,NaN
...,...,...,...,...,...
9995,Buzz Buzz a Diddle It,Matchbox,NaN,NaN,NaN
9996,Buzzer Beater,Jobii,NaN,NaN,NaN
9997,Buzzy Bee,The Beanies,NaN,NaN,NaN
9998,Buồn Thì Cứ Khóc Đi,Lynk Lee,NaN,NaN,NaN


In [8]:
final_df.to_csv('../data/merged_data.csv', index=False)

# Conclussions

After analyzing and performing exploratory data analysis (EDA) on each dataset, it's clear that combining these datasets was a significant challenge, especially when aiming to generate insightful data. 

An example of this is illustrating which songs were Grammy-nominated and winners and which were not, among other ways of presenting conclusions with this data.